<a href="https://colab.research.google.com/github/nishkarsh25/Sentence-Auto-Completion-using-Probability/blob/main/Auto_completion_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from urllib import request
from nltk.corpus import brown

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
url = "http://www.gutenberg.org/files/2554/2554-0.txt"
response = request.urlopen(url)
l = response.read().decode('utf8')

url1 = "https://www.gutenberg.org/cache/epub/27827/pg27827.txt"
response1 = request.urlopen(url1)
l+= response1.read().decode('utf8')

url2 = "https://www.gutenberg.org/cache/epub/203/pg203.txt"
response2 = request.urlopen(url2)
l+= response2.read().decode('utf8')

url3 = "https://www.gutenberg.org/files/55/55-0.txt"
response3 = request.urlopen(url3)
l+= response3.read().decode('utf8')

url4 = "https://www.gutenberg.org/files/10007/10007-0.txt"
response4 = request.urlopen(url4)
l+= response4.read().decode('utf8')

url5 = "https://www.gutenberg.org/cache/epub/779/pg779.txt"
response5 = request.urlopen(url5)
l+= response5.read().decode('utf8')
     
corpus = l.split()


lower_case_corpus = [w.lower() for w in corpus]
vocab = set(lower_case_corpus)

print('CORPUS EXAMPLE: ' + str(lower_case_corpus[:30]) + '\n\n')
print('VOCAB EXAMPLE: ' + str(list(vocab)[:10]))



CORPUS EXAMPLE: ['\ufeffthe', 'project', 'gutenberg', 'ebook', 'of', 'crime', 'and', 'punishment,', 'by', 'fyodor', 'dostoevsky', 'this', 'ebook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'in', 'the', 'united', 'states', 'and', 'most', 'other', 'parts', 'of', 'the']


VOCAB EXAMPLE: ['moody', 'starting.', "tortur'd", 'addressing,', 'what!”', 'legislating!”', 'confidence', 'bar’ls.”', 'proved', 'lace,']


In [ ]:
print('Total words in Corpus: ' + str(len(lower_case_corpus)))
print('Vocab of the Corpus: ' + str(len(vocab)))

Total words in Corpus: 549241
Vocab of the Corpus: 46819


In [ ]:
unigram_counts={}
bigram_counts = {}
trigram_counts = {}
quadgram_counts={}

for i in range(len(lower_case_corpus)):
      unigram= (lower_case_corpus[i])
      if unigram in unigram_counts.keys():
         unigram_counts[unigram] += 1
      else:
         unigram_counts[unigram] = 1
for i in range(len(lower_case_corpus) - 1):
      bigram = (lower_case_corpus[i], lower_case_corpus[i+1])
      if bigram in bigram_counts.keys():
         bigram_counts[bigram] += 1
      else:
         bigram_counts[bigram] = 1
for i in range(len(lower_case_corpus) - 2):
      trigram = (lower_case_corpus[i], lower_case_corpus[i+1], lower_case_corpus[i+2])
      if trigram in trigram_counts.keys():
         trigram_counts[trigram] += 1
      else:
         trigram_counts[trigram] = 1
for i in range(len(lower_case_corpus) - 3):     
      quadgram = (lower_case_corpus[i], lower_case_corpus[i+1], lower_case_corpus[i+2], lower_case_corpus[i+3])
      if quadgram in quadgram_counts.keys():
         quadgram_counts[quadgram] += 1
      else:
         quadgram_counts[quadgram] = 1
      
     
     
print("Example, count for unigram ('the') is: " + str(unigram_counts[('the')]))
print("Example, count for bigram ('the', 'king') is: " + str(bigram_counts[('the', 'king')]))
print("Example, count for trigram ('the', 'king' ,'of') is: " + str(trigram_counts[('the', 'king','of')]))







Example, count for unigram ('the') is: 26098
Example, count for bigram ('the', 'king') is: 47
Example, count for trigram ('the', 'king' ,'of') is: 15


In [ ]:
# Function takes sentence as input and suggests possible words that comes after the sentence  
def suggest_next_word(input_, bigram_counts, trigram_counts, vocab):
    # Consider the last bigram of sentence
    tokenized_input = word_tokenize(input_.lower())
    last_trigram= tokenized_input[-3:]
    last_bigram = tokenized_input[-2:]
    last_unigram= tokenized_input[-1:]
    
    # Calculating probability for each word in vocab
    vocab_probabilities = {}
    for vocab_word in vocab:
        test_quadgram = (last_trigram[0], last_trigram[1], last_trigram[2], vocab_word)
        test_trigram = (last_trigram[0], last_trigram[1], last_trigram[2])

        test_quadgram_count = quadgram_counts.get(test_quadgram, 0)
        test_trigram_count = trigram_counts.get(test_trigram, 0)
        
        if(test_trigram_count!=0):
           probability = test_quadgram_count / test_trigram_count
           vocab_probabilities[vocab_word] = probability
        
        else: 
          
            test_trigram = (last_bigram[0], last_bigram[1], vocab_word)
            test_bigram = (last_bigram[0], last_bigram[1])

            test_trigram_count = trigram_counts.get(test_trigram, 0)
            test_bigram_count = bigram_counts.get(test_bigram, 0)
           
            if(test_bigram_count!=0):
                probability = test_trigram_count / test_bigram_count
                vocab_probabilities[vocab_word] = probability
            else:
             
                test_bigram = (last_unigram[0], vocab_word)
                test_unigram = (last_unigram[0])

                test_bigram_count = bigram_counts.get(test_bigram, 0)
                test_unigram_count = unigram_counts.get(test_unigram, 0)
                
                if(test_unigram_count!=0):
                  probability = test_bigram_count / test_unigram_count
                  vocab_probabilities[vocab_word] = probability
                else:
                  vocab_probabilities[vocab_word] = 0

        
          

    
    # Sorting the vocab probability in descending order to get top probable words
    top_suggestions = sorted(vocab_probabilities.items(), key=lambda x: x[1], reverse=True)[:1]
    return top_suggestions[0][0]

    



In [ ]:
suggest_next_word('I am the king',bigram_counts, trigram_counts,  vocab)

[('of', 0.3191489361702128)]

In [ ]:
suggest_next_word('I am the king of', bigram_counts, trigram_counts, vocab)


[('the', 0.6)]

In [ ]:
suggest_next_word('I am the king of france ', bigram_counts, trigram_counts, vocab)


[('and', 0.6666666666666666)]

In [ ]:
testfile = open("test_words.txt", "r")

for aline in testfile:
  values=aline.split()
  s=values[0]+' '+values[1]+' '+values[2]+' '+values[3]
  for i in range(3):
   out=suggest_next_word(s, bigram_counts, trigram_counts, vocab)
   s=s+' '+out;
  print(s)


1. Practice makes a massacre in paris,
2. Like share and the other is
3. The list has been a peculiar
4. She trekked towards the door. .
5. Honesty is the same at the
6. I am short and stout and
7. Thinking of Sanskrit and annotated by
8. This is a very different matter
9. Plants have a great deal of
10. No is a great tree, moody
11. Happy neurons generated moody and, moody
12. No more excuses to your respected
13. Coming all the terms of this
14. Library with five hundred dollars’ moody
15. And then there was the same
16. A sketcher in the same way
17. A body in many places. .
18. That's the reason of this is,
19. My birthday is a great tree,
20. I like this to a lady
21. What a lovely lady or a
22. I would rather be quite independent,
23. I want to give some of
24. Do you like to do everything
25. Put the toys of every description,
26. Let us go and inform the
27. She found a boy who was
28. The ball is over.” moody and,
29. I have three little children and
30. Please pass the night 